# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import requests
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = "../output_data/weather_data.csv"

data = pd.read_csv(path)
data.head()

,Unnamed: 0,City,Latitude,Longtitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hong Kong,22.29,114.16,297.04,78,20,1.50,HK,1586997015
1,1,Ahipara,-35.17,173.17,291.28,77,29,8.45,NZ,1586997521
2,2,Busselton,-33.65,115.33,287.04,97,79,3.25,AU,1586997165
3,3,Rikitea,-23.12,-134.97,298.29,72,100,3.42,PF,1586997166
4,4,Sawākin,19.11,37.33,295.20,54,0,3.53,SD,1586997685


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
locations = data[['Latitude','Longtitude']]
weights = data['Humidity']


In [5]:
heatmap_layer = gmaps.heatmap_layer(locations,weights=weights)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* A max temperature lower than 80 degrees but higher than 70.
  * Wind speed less than 10 mph.
  * Zero cloudiness.

In [6]:
temp= (data['Max Temperature']<270) &(data['Max Temperature']>250)
windspeed = data['Wind Speed'] < 10
cloudiness= data['Cloudiness'] == 0
ideal_weather = data[temp & windspeed & cloudiness]
ideal_weather

,Unnamed: 0,City,Latitude,Longtitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,Talnakh,69.49,88.40,261.15,55,0,4.00,RU,1586997142
180,180,Khatanga,71.97,102.50,252.75,90,0,3.22,RU,1586997192
192,192,Pangody,65.85,74.49,262.16,90,0,4.93,RU,1586997735


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather.assign(hotelname="")
hotel_df

,Unnamed: 0,City,Latitude,Longtitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,hotelname
24,24,Talnakh,69.49,88.40,261.15,55,0,4.00,RU,1586997142,
180,180,Khatanga,71.97,102.50,252.75,90,0,3.22,RU,1586997192,
192,192,Pangody,65.85,74.49,262.16,90,0,4.93,RU,1586997735,


In [8]:
target_search = "hotel"
target_radius = 5000
params = {#"location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "key": g_key}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longtitude']
    target_coordinates = f"{lat},{lng}"
    params["location"] = target_coordinates  
    
    #print(f"Retrieving Results for Index {index}: {target_coordinates}.")
    response = requests.get(base_url, params=params).json()
       
  
    #data['results'][0]['name']  
    try:
        print(f"Closest {target_coordinates} hotel is {response['results'][0]['name']}.") 
        hotel_df.loc[index,'hotelname'] = response['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df
    

Closest 69.49,88.4 hotel is Talnakh Hotel.
Missing field/result... skipping.
Closest 65.85,74.49 hotel is Otel' "Medvezh'ye".


,Unnamed: 0,City,Latitude,Longtitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,hotelname
24,24,Talnakh,69.49,88.40,261.15,55,0,4.00,RU,1586997142,Talnakh Hotel
180,180,Khatanga,71.97,102.50,252.75,90,0,3.22,RU,1586997192,
192,192,Pangody,65.85,74.49,262.16,90,0,4.93,RU,1586997735,"Otel' ""Medvezh'ye"""


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotelname}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longtitude"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))